In [43]:
couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
  "name": "bedroom 1",
  "type": "room",
}
queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}
door_b = {
    "name": "door b",
    "type": "door",
}
key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

door_c = {
    "name": "door c",
    "type": "door",
}
bedroom_2 = {
  "name": "bedroom 2",
  "type": "room",
}
double_bed = {
    "name": "double bed",
    "type": "furniture",
}
dresser = {
    "name": "dresser",
    "type": "furniture",

}
key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}
door_d = {
    "name": "door d",
    "type": "door",
}
key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}
living_room = {
  "name": "living room",
  "type": "room",
}
dining_table = {
    "name": "dining table",
    "type": "furniture",
}
outside = {
  "name": "outside"
}


In [44]:
all_rooms = [game_room, bedroom_1,bedroom_2,living_room,outside]

In [45]:
all_doors = [door_a, door_b, door_c,door_d]

In [46]:
""" Map of game """

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed, door_a, door_b, door_c],
    "bedroom 2": [double_bed, dresser, door_b],
    "living room": [dining_table, door_d],
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "outside": [door_a],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [living_room, bedroom_1],
    "door d": [living_room, outside]
}

choices = {
    '1': "Examine items",
    '2': "Colectables"
} 

In [47]:
""" Game function """

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [48]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print(
        "You wake up on a couch and find yourself in a strange house with no windows "
        "which you have never been to before.\n"
        "You don't remember why you are here and what had happened before.\n"
        "You feel some unknown danger is approaching and you must get out of the house, NOW!"
    )    
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
        
    else:
        explore_room(room)
        #choices_list = [f'{keys} : {values}' for keys, values in choices.items()]
        choices_list = ', '.join(map(lambda key: f'{key[0]}: {key[1]} ', choices.items()))
        print("Choices " + choices_list)
        print("You are now in " + room["name"])
        intended_action = input("Your choice?  ").strip()
        match intended_action:
            case "1":
                intended_action = input("What would you like to examine?  ").strip()
                examine_item(intended_action)
            case "2":
                check_item()
            case _:
                print("This is not a choice.")
        #22linebreak()
        
    if game_state["current_room"] != game_state["target_room"]:
        play_room(room) 

def check_item():
    """
    List all items belonging to you.
    """
    #print(type(game_state["keys_collected"]))
    if not game_state["keys_collected"]:
        print("You colected 0")
    else:
        items = [ i["name"] for i in game_state["keys_collected"] if i["type"] == "key" ]
        print("You colected " + ", ".join(items))

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("This is the " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door_name, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door_name["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def unlock_door(key_list, current_room, door_name):
    
    next_room = current_room
    output = "It is locked but you don't have the key."  
    
    for key in key_list:
        if key["target"] == door_name:  
            
            print("You unlock it with a key you have.")
            next_room = get_next_room_of_door(door_name, current_room)
            
            if input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes':
                play_room(next_room)
                return ""  
            else:
                return ""  
    print(output)
    return output  


def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    output = None
    
    for item in object_relations[current_room["name"]]:
           if(item["name"] == item_name):
            output = "You examine the " + item_name + ". "
            if(item["type"] == "door"):
                output = unlock_door(game_state["keys_collected"], current_room, item)                
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                    
                    print(output)
                    
                                  
                else:
                    output += "There isn't anything interesting about it."
                    print(output)
                                    
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
        #play_room(current_room)

    

In [49]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before.
You don't remember why you are here and what had happened before.
You feel some unknown danger is approaching and you must get out of the house, NOW!
This is the game room. You find couch, piano, door a
Choices 1: Examine items , 2: Colectables 
You are now in game room
This is not a choice.
This is the game room. You find couch, piano, door a
Choices 1: Examine items , 2: Colectables 
You are now in game room


KeyboardInterrupt: Interrupted by user